## Домашнє завдання: Оцінка та моніторинг

У цьому домашньому завданні ми оцінюватимемо якість нашої системи RAG.

> Можливо, ваші відповіді не будуть точно співпадати. Якщо це так, виберіть найближчу.
Рішення:

* Відео: TBA
* Нотатник: TBA

## Отримання даних

Почнемо з отримання набору даних. Ми використаємо дані, які ми згенерували в модулі.

Зокрема, ми оцінюватимемо якість нашої системи RAG
за допомогою [gpt-4o-mini](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv)

Прочитайте його:

```python
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
```

Ми будемо використовувати лише перші 300 документів:

```python
df = df.iloc[:300]
```

## Питання 1. Отримання моделі векторних представлень

Тепер отримайте модель векторних представлень `multi-qa-mpnet-base-dot-v1` з
[бібліотеки Sentence Transformer](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

> Примітка: це не та ж модель, що в домашньому завданні 3
```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Створіть векторні представлення для першої відповіді LLM:

```python
answer_llm = df.iloc[0].answer_llm
```

Яке перше значення отриманого вектора?

* -0.42
* -0.22
* -0.02
* 0.21

## Питання 2. Обчислення скалярного добутку

Тепер для кожної пари відповідей створимо векторні представлення та обчислимо скалярний добуток між ними

Ми розмістимо результати (оцінки) у списку `evaluations`

Який 75-й перцентиль оцінок?

* 21.67
* 31.67
* 41.67
* 51.67

## Питання 3. Обчислення косинуса

З питання 2 ми бачимо, що результати не знаходяться в діапазоні [0, 1]. Це тому, що вектори, отримані з цієї моделі, не нормалізовані.

Тому нам потрібно їх нормалізувати.

Для цього ми

* Обчислимо норму вектора
* Поділимо кожен елемент на цю норму

Отже, для вектора `v`, це буде `v / ||v||`

У numpy це робиться так:

```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Давайте вставимо це у функцію, а потім обчислимо скалярний добуток 
між нормалізованими векторами. Це дасть нам косинусну схожість.

Який 75-й перцентиль косинусної схожості в оцінках?

* 0.63
* 0.73
* 0.83
* 0.93

## Питання 4. ROUGE

Тепер ми розглянемо альтернативну метрику - оцінку ROUGE.  

Це набір метрик, який порівнює дві відповіді на основі збігу n-грам, послідовностей слів та пар слів.

Вона може дати більш детальну оцінку схожості текстів, ніж просто косинусна схожість.

Нам не потрібно реалізовувати це самостійно, є python-пакет для цього:

```bash
pip install rouge
```

(Остання версія на момент написання - `1.0.1`)

Давайте обчислимо оцінку ROUGE між відповідями на індексі 10 нашого датафрейму (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

Існують три оцінки: `rouge-1`, `rouge-2` та `rouge-l`, і точність, повнота та F1 оцінка для кожної з них.

* `rouge-1` - збіг уніграмів,
* `rouge-2` - біграмів,
* `rouge-l` - найдовша спільна підпослідовність

Яка F оцінка для `rouge-1`?

- 0.35
- 0.45
- 0.55
- 0.65

## Питання 5. Середня оцінка ROUGE

Давайте обчислимо середнє значення між `rouge-1`, `rouge-2` та `rouge-l` для того ж запису з питання 4

- 0.35
- 0.45
- 0.55
- 0.65

## Питання 6. Середня оцінка ROUGE для всіх точок даних

Тепер давайте обчислимо оцінку для всіх записів

```python
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
```

І створимо з них датафрейм

Яке середнє значення `rouge_l` серед усіх записів?

- 0.10
- 0.20
- 0.30
- 0.40

Getting the data

In [2]:

import pandas as pd

# Define the URL for the dataset
# github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
# url = f'{github_url}?raw=1'

# Load the dataset
df = pd.read_csv('./data/results-gpt4o-mini.csv')

# Use only the first 300 documents
df = df.iloc[:300]


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   answer_llm   300 non-null    object
 1   answer_orig  300 non-null    object
 2   document     300 non-null    object
 3   question     300 non-null    object
 4   course       300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB


In [10]:
df.head(10)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
5,"The course videos are pre-recorded, and you ca...","The course videos are pre-recorded, you can st...",39fda9f0,Are the course videos live or pre-recorded?,machine-learning-zoomcamp
6,You can start watching the course videos right...,"The course videos are pre-recorded, you can st...",39fda9f0,When can I start watching the course videos?,machine-learning-zoomcamp
7,"Yes, the live office hours sessions are recorded.","The course videos are pre-recorded, you can st...",39fda9f0,Are the live office hours sessions recorded?,machine-learning-zoomcamp
8,You can find the office hours sessions in the ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I find the office hours sessions?,machine-learning-zoomcamp
9,You can access the pre-recorded course videos ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I access the pre-recorded course vid...,machine-learning-zoomcamp


## Q1: Getting the embeddings model

In [14]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
embedding_model.save('./model/multi-qa-mpnet-base-dot-v1')

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





KeyboardInterrupt: 

In [3]:
# Загрузка модели из локальной папки
embedding_model = SentenceTransformer('./model/multi-qa-mpnet-base-dot-v1')

# Create the embeddings for the first LLM answer
answer_llm = df.iloc[0].answer_llm
embedding = embedding_model.encode(answer_llm)

# Print the first value of the resulting vector
print("First value of the resulting vector:", embedding[0])


First value of the resulting vector: 0.0643027


## Q2: Computing the dot product

In [4]:

import numpy as np

# Initialize an empty list for the evaluations
evaluations = []

# Compute embeddings and dot products
for _, row in df.iterrows():
    emb_llm = embedding_model.encode(row['answer_llm'])
    emb_orig = embedding_model.encode(row['answer_orig'])
    dot_product = np.dot(emb_llm, emb_orig)
    evaluations.append(dot_product)

# Compute the 75th percentile of the scores
percentile_75 = np.percentile(evaluations, 75)
print("75th percentile of the dot product scores:", percentile_75)


75th percentile of the dot product scores: 0.8139468133449554


## Q3: Computing the cosine

In [5]:

# Function to normalize a vector
def normalize(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

# Compute cosine similarity
cosine_similarities = []
for _, row in df.iterrows():
    emb_llm = normalize(embedding_model.encode(row['answer_llm']))
    emb_orig = normalize(embedding_model.encode(row['answer_orig']))
    cosine_similarity = np.dot(emb_llm, emb_orig)
    cosine_similarities.append(cosine_similarity)

# Compute the 75th percentile of the cosine similarities
percentile_75_cosine = np.percentile(cosine_similarities, 75)
print("75th percentile of the cosine similarities:", percentile_75_cosine)


75th percentile of the cosine similarities: 0.8139468133449554


## Q4: Rouge

In [11]:
from rouge import Rouge

# Initialize ROUGE scorer
rouge_scorer = Rouge()

# Get the record with doc_id=5170565b
record = df[df['document'] == '5170565b'].iloc[0]

# Compute ROUGE scores
scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]

# Print the F1 score for ROUGE-1
rouge_1_f1 = scores['rouge-1']['f']
print("F1 score for ROUGE-1:", rouge_1_f1)

F1 score for ROUGE-1: 0.45454544954545456


## Q5: Compute the average ROUGE score for the same record

In [12]:
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']
rouge_avg = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3
print("Average ROUGE score for the record:", rouge_avg)


Average ROUGE score for the record: 0.35490034990035496


## Q6: Average rouge score for all the data points

In [13]:

# Compute ROUGE scores for all records
rouge_scores = []

for _, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    rouge_1_f1 = scores['rouge-1']['f']
    rouge_2_f1 = scores['rouge-2']['f']
    rouge_l_f1 = scores['rouge-l']['f']
    rouge_avg = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3
    rouge_scores.append(rouge_l_f1)

# Compute the average ROUGE-L score across all records
average_rouge_l = np.mean(rouge_scores)
print("Average ROUGE-L score across all records:", average_rouge_l)


Average ROUGE-L score across all records: 0.3538074656078652
